# Xenium Mouse PE Aging Spatial Transcriptomic Analysis
This notebook reproduces Figures 2A–2E. Data files under `./data/`, results saved to `./results/`.

## 0. Setup and Imports

In [ ]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt

# Create results directory
os.makedirs('./results', exist_ok=True)

## 1. Load Processed AnnData Objects

In [ ]:
# Paths to processed .h5ad files in the data folder
adata_paths = {
    'Control-Young': './data/Control-Young.h5ad',
    'Control-Aged':  './data/Control-Aged.h5ad',
    'PE-Young':      './data/PE-Young.h5ad',
    'PE-Aged':       './data/PE-Aged.h5ad',
}

# Load each dataset and concatenate
adatas = {k: sc.read_h5ad(v) for k, v in adata_paths.items()}
adata_all = sc.concat(
    list(adatas.values()),
    label='condition',
    keys=list(adatas.keys()),
    join='outer'
)
print(adata_all)

## 2. Figure 2A: Marker Gene Dotplot

In [ ]:
# Marker genes by cell type
marker_genes = {
    "Cardiomyocyte": ["Ryr2", "Myl2", "Nppb", "Myh7"],
    "Fibroblast":    ["Fstl1", "Col1a1", "Postn"],
    "Endothelial":   ["Cd36", "Pecam1", "Vwf", "Emcn"],
    "Epicardial":    ["Gpx3"],
    "Smooth muscle": ["Myh11", "Acta2"],
    "Macrophage":    ["C1qa", "Cd68"],
    "Lymphocyte":    ["Ptprc"],
    "Pericyte":      ["Rgs5"],
}

# Dotplot
sc.pl.dotplot(
    adata_all,
    var_names=marker_genes,
    groupby='condition',
    use_raw=False,
    standard_scale='var',
    figsize=(10, 6),
    show=False
)
fig = plt.gcf()
fig.savefig('./results/Figure2A_dotplot.png', dpi=300, bbox_inches='tight')
plt.close(fig)

## 3. Figure 2B: Spatial Scatter of Final Cell Types

In [ ]:
# Define colors
all_types = sorted(adata_all.obs['final_cell_type'].unique())
custom_colors = {
    "Cardiomyocyte": "#e41a1c",
    "Endothelial":   "#377eb8",
    "Fibroblast":    "#4daf4a",
    "Smooth muscle": "#e65100",
}
base_colors = plt.cm.tab20.colors
color_dict = {**custom_colors}
for i, ct in enumerate([ct for ct in all_types if ct not in custom_colors]):
    color_dict[ct] = base_colors[i % len(base_colors)]

# Plot
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()
labels = ['Control-Young', 'Control-Aged', 'PE-Young', 'PE-Aged']

for ax, cond in zip(axes, labels):
    sub = adata_all[adata_all.obs['condition'] == cond]
    coords = sub.obsm['spatial']
    types = sub.obs['final_cell_type']
    for ct in all_types:
        mask = types == ct
        if mask.any():
            ax.scatter(coords[mask, 0], coords[mask, 1],
                       s=0.1, color=color_dict[ct], label=ct)
    ax.set_title(cond, fontsize=20)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect('equal')

# Global legend
handles = [plt.Line2D([0], [0], marker='o', color='w',
                      label=ct, markerfacecolor=color_dict[ct], markersize=6)
           for ct in all_types]
fig.legend(handles=handles, title='Final Cell Type',
           loc='lower center', ncol=4, fontsize=12, title_fontsize=14)
plt.subplots_adjust(bottom=0.15, top=0.95, wspace=0.1, hspace=0.1)
fig.savefig('./results/Figure2B_spatial_scatter.png', dpi=300, bbox_inches='tight')
plt.close(fig)

## 4. Figures 2C–2E: Gene Expression Overlays

In [ ]:
from matplotlib.lines import Line2D

def plot_overlay(gene, color, fname):
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    axes = axes.flatten()
    for ax, cond in zip(axes, labels):
        sub = adata_all[adata_all.obs['condition'] == cond]
        coords = sub.obsm['spatial']
        expr = sub[:, gene].X.toarray().flatten()
        # Plot no-expression
        ax.scatter(coords[expr == 0, 0], coords[expr == 0, 1],
                   s=0.1, c='lightgray', alpha=0.8)
        # Plot expressed
        ax.scatter(coords[expr > 0, 0], coords[expr > 0, 1],
                   s=0.1, c=color, alpha=0.8)
        ax.set_title(cond, fontsize=18)
        ax.set_xticks([]); ax.set_yticks([]); ax.set_aspect('equal')
    # Legend
    legend_elems = [
        Line2D([0], [0], marker='o', color='w', markerfacecolor='lightgray', markersize=6, label='No Expression'),
        Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=6, label=gene)
    ]
    fig.legend(handles=legend_elems, title=f'{gene} Expression',
               loc='upper center', ncol=2, fontsize=12, title_fontsize=14)
    plt.subplots_adjust(top=0.9, bottom=0.05, wspace=0.1, hspace=0.1)
    fig.savefig(f'./results/{fname}', dpi=300, bbox_inches='tight')
    plt.close(fig)

# Generate overlays
plot_overlay('Ryr2', 'orange', 'Figure2C_Ryr2_overlay.png')
plot_overlay('Col1a1', 'green',  'Figure2D_Col1a1_overlay.png')
plot_overlay('Cd68',  'purple', 'Figure2E_Cd68_overlay.png')